In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, LeakyReLU
from keras_self_attention import SeqSelfAttention
from keras.models import Sequential
from keras.metrics import Precision, Recall, BinaryAccuracy
import datetime

2023-03-08 16:30:45.651371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU and enable memory growth
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        # print(len(gpus), "Physical GPUs:", len(logical_gpus), "Logical GPU")
        print(f'Physical GPUs: {gpus} Logical GPUs: {logical_gpus}')
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print('Error: ', e)
else:
    print('No GPUs found')


Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] Logical GPUs: [LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2023-03-08 09:58:20.424445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 09:58:20.821050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4646 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5


In [16]:
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 1.0
# sess = tf.compat.v1.Session(config=config)


<class 'tensorflow.python.data.ops.dataset_ops.ParallelMapDataset'>


In [ ]:
# Estimated time: 1.5 min
train = tf.keras.utils.image_dataset_from_directory(
    '/coding-drive/DATASETS/face-recognition-tensorflow/', 
    batch_size=4,
    image_size=(224, 224),
    color_mode='grayscale',
    shuffle=True
    )

In [ ]:
test = tf.keras.utils.image_dataset_from_directory(
    '/coding-drive/DATASETS/face-recognition-tensorflow-test-data/',
    batch_size=4,
    image_size=(224, 224),
    color_mode='grayscale',
    shuffle=True
)


## VISUALIZATION OF THE PIPELINE

In [ ]:
# data_iter = data.as_numpy_iterator()

In [ ]:
# batch = data_iter.next()
# for i in range(4):
#     plt.subplot(2, 2, i+1)
#     plt.imshow(batch[0][i].reshape(224, 224), cmap='gray')
#     plt.title(batch[1][i])
#     plt.axis('off')

## Preprocessing

In [ ]:
train = train.map(lambda x,y: (x/255, y))
test = test.map(lambda x,y: (x/255, y))

## Splitting the data

In [ ]:
# train_size = int(len(data)*.9)
# val_size = int(len(data)*.2)
# test_size = int(len(data)*.1)


In [ ]:
# train = data.take(train_size)
# test = data.skip(train_size).take(test_size)
# test = data.skip(train_size+val_size).take(test_size)


## Building the model

In [2]:
model = Sequential()
model.add(Input(shape=(224, 224, 1), name='input'))

# Convolutional layer - 1 (32 filters, 3x3 kernel, relu activation)
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv2d_1'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_1'))
model.add(BatchNormalization(name='batch_norm_1'))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv2d_2'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_2'))
model.add(BatchNormalization(name='batch_norm_2'))

model.add(SeqSelfAttention(name='attention_1'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer - 2 (64 filters, 3x3 kernel, relu activation)
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv2d_3'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_3'))
model.add(BatchNormalization(name='batch_norm_3'))

model.add(Conv2D(filters=64, kernel_size=(3, 3),padding='same', name='conv2d_4'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_4'))
model.add(BatchNormalization(name='batch_norm_4'))

model.add(SeqSelfAttention(name='attention_2'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer - 3 (128 filters, 3x3 kernel, relu activation)
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', name='conv2d_5'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_5'))
model.add(BatchNormalization(name='batch_norm_5'))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', name='conv2d_6'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_6'))
model.add(BatchNormalization(name='batch_norm_6'))

model.add(SeqSelfAttention(name='attention_3'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer - 4 (256 filters, 3x3 kernel, relu activation)
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', name='conv2d_7'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_7'))
model.add(BatchNormalization(name='batch_norm_7'))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', name='conv2d_8'))
model.add(LeakyReLU(alpha=0.1, name='leaky_relu_8'))
model.add(BatchNormalization(name='batch_norm_8'))

model.add(SeqSelfAttention(name='attention_4'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Dense layer - 1 (512 neurons, relu activation)
model.add(Dense(512, activation='relu', name='dense_1'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', name='dense_2'))
model.add(Dropout(0.2))

# Dense layer - 2 (128 neurons, relu activation)
model.add(Dense(128, activation='relu', name='dense_3'))
model.add(Dropout(0.2))

# Dense layer - output (3 neurons, softmax activation)
model.add(Dense(3, activation='softmax', name='output'))


2023-03-08 16:30:58.689290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 16:30:59.692782: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4646 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.wa

ValueError: Exception encountered when calling layer "attention_1" (type SeqSelfAttention).

in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras_self_attention/seq_self_attention.py", line 158, in call  *
        e = self._call_additive_emission(inputs)
    File "/usr/local/lib/python3.10/dist-packages/keras_self_attention/seq_self_attention.py", line 195, in _call_additive_emission  *
        q = K.expand_dims(K.dot(inputs, self.Wt), 2)
    File "/usr/local/lib/python3.10/dist-packages/keras/backend.py", line 2450, in dot
        tf.matmul(xt, yt), x_shape[:-1] + y_shape[:-2] + y_shape[-1:]

    ValueError: Dimensions must be equal, but are 32 and 224 for '{{node attention_1/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](attention_1/Reshape, attention_1/Reshape_1)' with input shapes: [?,32], [224,32].


Call arguments received by layer "attention_1" (type SeqSelfAttention):
  • inputs=tf.Tensor(shape=(None, 224, 224, 32), dtype=float32)
  • mask

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

## Training the model

In [ ]:
model_name = input('Enter model name: ')

In [ ]:
history = model.fit(
    train, 
    epochs=10, 
    validation_data=test, 
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            log_dir=f"training_output/{model_name}/tensorboard/fit/" + datetime.datetime.now().strftime("%m-%d_%H-%M-%S"),
            histogram_freq=1
        ),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'training_output/{model_name}/checkpoints/'+'epoch_{epoch: 02d}_val_loss_{val_loss: .2f}.hdf5',
            save_freq='epoch',
            save_best_only=False,
            save_weights_only=False,
            monitor='val_loss',
            ),
        tf.keras.callbacks.ReduceLROnPlateau('val_loss', patience=2, verbose=1),
        tf.keras.callbacks.CSVLogger(f'training_output/{model_name}/training_log.csv'),
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10**(epoch / 20)),
        ]
    )

In [ ]:
model.save(f'training_output/{model_name}/final_model.h5')

## Plotting the results

In [ ]:
history.history

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['sparse_categorical_accuracy'],
         color='teal', label='accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'],
         color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# pre = Precision()
# re = Recall()
# acc = BinaryAccuracy()

In [ ]:
# batch = test_iter.next()
# for i in range(4):
#     plt.subplot(2, 2, i+1)
#     plt.imshow(batch[0][i].reshape(224, 224, 1), cmap='gray')
#     plt.title(batch[1][i])
#     plt.axis('off')
#     print(
#         f'Predicted: {model.predict(batch[0][i].reshape(1, 224, 224, 1))[0].argmax()} Actual: {batch[1][i]}')
# plt.show()

In [ ]:
# model_test.predict(test)


In [ ]:
# for batch in test.as_numpy_iterator(): 
#     img_batch, label_batch = batch
#     yhat = model.predict(img_batch)
#     pre.update_state(label_batch, yhat)
#     re.update_state(label_batch, yhat)
#     acc.update_state(label_batch, yhat)


In [ ]:
# print(pre.result(), re.result(), acc.result())